In [ ]:
import sys
import os
import torch
import random
import numpy as np
import glob
import json
import cv2
import pandas as pd
from time import time
from PIL import Image

# Thêm đường dẫn hiện tại để import được các class từ file predict.py
sys.path.append('/code')

# Import các class từ code của bạn (Giả sử file code của bạn tên là predict.py)
# Lưu ý: Bạn cần đảm bảo file predict.py đã xóa đoạn login huggingface và sửa đường dẫn load model
from final4_optimized import Config, SimilarityModel, ObjectAwareModel, Sam, TinyViT, PromptEncoder, MaskDecoder, TwoWayTransformer, SamPredictor, associate_detections_to_trackers, KalmanBoxTracker

# Hàm cố định seed (Bắt buộc)
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("✅ Environment setup & Seed fixed.")

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Cấu hình đường dẫn cho môi trường Docker
class DockerConfig(Config):
    def __init__(self, video_id):
        super().__init__(video_id)
        # GHI ĐÈ CÁC ĐƯỜNG DẪN CHO DOCKER
        self.data_base = "/data/samples"  # Folder chứa data test được mount vào
        self.segment_base = "/segment_objects" # Folder chứa template images
        self.results_base = "/result"     # Folder output
        
        self.video_path = os.path.join(self.data_base, video_id, "drone_video.mp4")
        self.template_img_dir = os.path.join(self.segment_base, video_id, "original_images")
        self.template_mask_dir = os.path.join(self.segment_base, video_id, "mask_images")
        
        # Cấu hình Model (Load từ Local - QUAN TRỌNG: Phải tải weight về bỏ vào folder trong docker)
        self.dino_model_id = "./weight/DINO" # Sửa lại path này trỏ tới feature extractor local
        self.sam_checkpoint = './weight/mobile_sam.pt'
        self.yolo_model = './weight/ObjectAwareModel.pt'
        
        # Tạo folder kết quả nếu chưa có
        os.makedirs(self.results_base, exist_ok=True)

# Khởi tạo một Config giả để load model trước (video_id tạm)
dummy_cfg = DockerConfig("dummy")

print("⏳ Loading Models...")
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Load Similarity Model (DINOv3)
sim_model = SimilarityModel(dummy_cfg)

# 2. Load YOLO
yolo = ObjectAwareModel(dummy_cfg.yolo_model)

# 3. Load SAM
sam = Sam(image_encoder=TinyViT(img_size=1024, in_chans=3, num_classes=1000,
                                embed_dims=[64,128,160,320], depths=[2,2,6,2],
                                num_heads=[2,4,5,10], window_sizes=[7,7,14,7]),
          prompt_encoder=PromptEncoder(embed_dim=256, image_embedding_size=(64,64),
                                       input_image_size=(1024,1024), mask_in_chans=16),
          mask_decoder=MaskDecoder(num_multimask_outputs=3,
                                   transformer=TwoWayTransformer(depth=2, embedding_dim=256, mlp_dim=2048, num_heads=8),
                                   transformer_dim=256))

if os.path.exists(dummy_cfg.sam_checkpoint):
    sam.load_state_dict(torch.load(dummy_cfg.sam_checkpoint, map_location=device), strict=False)
else:
    print(f"❌ Warning: SAM checkpoint not found at {dummy_cfg.sam_checkpoint}")

sam.to(device).eval()
predictor = SamPredictor(sam)

print("✅ All Models Loaded Successfully!")

Model loaded successfully


In [ ]:
# Lấy danh sách các folder video trong thư mục /data/samples
# Cấu trúc giả định: /data/samples/VideoID/drone_video.mp4
search_path = "/data/samples/*"
test_cases = [os.path.basename(p) for p in glob.glob(search_path) if os.path.isdir(p)]

print(f"Found {len(test_cases)} test cases: {test_cases}")

all_predicted_time = []
all_results_json = {} # Dictionary để lưu kết quả gộp

for video_id in test_cases:
    print(f"▶️ Processing {video_id}...")
    
    # --- BẮT ĐẦU ĐO THỜI GIAN ---
    t1 = time()
    
    # 1. Setup Config riêng cho video này
    cfg = DockerConfig(video_id)
    
    # 2. Load Templates (Tính vào thời gian xử lý vì mỗi video template khác nhau)
    # Lưu ý: Nếu template load thất bại thì skip
    try:
        sim_model.load_templates(cfg.template_img_dir, cfg.template_mask_dir)
    except Exception as e:
        print(f"Skipping {video_id} due to template error: {e}")
        # Vẫn ghi thời gian dù lỗi để không bị crash chấm điểm
        t2 = time()
        predicted_time = int((t2 - t1) * 1000)
        all_predicted_time.append({"id": video_id, "answer": "error", "time": predicted_time})
        continue

    # 3. Video Setup
    cap = cv2.VideoCapture(cfg.video_path)
    W, H = int(cap.get(3)), int(cap.get(4))
    
    # Reset Trackers
    trackers = []
    KalmanBoxTracker.count = 0
    final_predictions = []
    frame_idx = 0 # Bắt đầu từ 0 hoặc đọc từ cfg.start_time nếu cần logic cắt video
    
    # Loop Frames
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        # --- LOGIC CHÍNH (Copy từ main loop của bạn) ---
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # YOLO Detect
        results = yolo(rgb, conf=cfg.conf_thres, verbose=False)
        boxes = results[0].boxes.xyxy.cpu().numpy() if results and results[0].boxes is not None else []
        
        high_score_candidates = []
        if len(boxes) > 0:
            predictor.set_image(rgb)
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                # Check area ratio
                if not (cfg.min_area_ratio <= (x2-x1)*(y2-y1)/(W*H) <= cfg.max_area_ratio): continue

                masks, _, _ = predictor.predict(box=box, multimask_output=False)
                if len(masks) == 0: continue
                mask = masks[0]
                
                # Crop & Feature
                yy, xx = np.where(mask)
                if len(yy) == 0: continue
                y1b, y2b, x1b, x2b = yy.min(), yy.max()+1, xx.min(), xx.max()+1
                crop = rgb[y1b:y2b, x1b:x2b]
                crop_mask = mask[y1b:y2b, x1b:x2b]
                
                feat = sim_model.extract_features(Image.fromarray(crop), crop_mask)
                s1, s2, s3, avg_match, app_bonus = sim_model.compute_scores(feat)
                size_pen = sim_model.size_penalty([x1b, y1b, x2b, y2b], W * H)
                final_score = 0.7 * avg_match + 0.25 * app_bonus + 0.05 * size_pen
                
                if final_score > cfg.SCORE_THRESHOLD:
                    high_score_candidates.append({
                        'bbox': np.array([x1b, y1b, x2b, y2b]), 
                        'score': final_score
                    })
        
        # SORT Tracking
        if len(high_score_candidates) > 0:
            dets_for_track = np.array([c['bbox'] for c in high_score_candidates])
        else:
            dets_for_track = np.empty((0, 4))
            
        trks_for_track = np.zeros((len(trackers), 4))
        to_del = []
        for t, trk in enumerate(trackers):
            pos = trk.predict()[0]
            trks_for_track[t, :] = [pos[0], pos[1], pos[2], pos[3]]
            if np.any(np.isnan(pos)): to_del.append(t)
        for t in reversed(to_del): trackers.pop(t)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets_for_track, trks_for_track, iou_threshold=cfg.IOU_THRESHOLD)

        for m in matched:
            trackers[m[1]].update(dets_for_track[m[0]], score=high_score_candidates[m[0]]['score'])
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets_for_track[i])
            trk.update(dets_for_track[i], score=high_score_candidates[i]['score'])
            trackers.append(trk)

        # Get Best Tracker
        active_trackers = []
        for i, trk in enumerate(trackers):
             # Logic visualize/save của bạn: hit_streak >= MIN_HITS or frame mới
             if (trk.time_since_update < 1) and (trk.hit_streak >= cfg.MIN_HITS or frame_idx <= 5): 
                active_trackers.append({"bbox": trk.get_state()[0], "score": trk.last_score, "id": trk.id})
        
        best_obj = max(active_trackers, key=lambda x: x['score']) if active_trackers else None
        
        # Save Frame Result
        frame_res = {
            "frame_id": frame_idx,
            "box": best_obj['bbox'].astype(int).tolist() if best_obj else [],
            "score": float(best_obj['score']) if best_obj else 0.0,
            "track_id": int(best_obj['id']) if best_obj else -1
        }
        final_predictions.append(frame_res)
        frame_idx += 1
        
    cap.release()
    # --- KẾT THÚC ĐO THỜI GIAN ---
    t2 = time()
    
    # Tính toán output
    predicted_time = int((t2 - t1) * 1000) # miliseconds
    
    # Lưu kết quả vào list tổng
    all_predicted_time.append({"id": video_id, "answer": "processed", "time": predicted_time})
    
    # Lưu json prediction riêng cho từng video vào object tổng (tuỳ format đề bài, thường là list các object)
    all_results_json[video_id] = final_predictions

# --- GHI FILE OUTPUT CUỐI CÙNG ---
output_dir = "/result"
os.makedirs(output_dir, exist_ok=True)

# 1. File time_submission.csv
df_time = pd.DataFrame(all_predicted_time)
df_time.to_csv(os.path.join(output_dir, "time_submission.csv"), index=False)

# 2. File jupyter_submission.json
# Format đề bài yêu cầu thường là list các video hoặc dictionary. 
# Ở đây tôi dump cả cục dictionary chứa tất cả video.
with open(os.path.join(output_dir, "jupyter_submission.json"), 'w') as f:
    json.dump(all_results_json, f, indent=2)

print("\n✅ DONE! Output saved to /result/")
print(df_time)